**Duración aproximada: 9 minutos**

# Módulo 2: Scraping con Selenium
## LATAM Airlines
<a href="https://www.latam.com/es_ar/"><img src="https://i.pinimg.com/originals/dd/52/74/dd5274702d1382d696caeb6e0f6980c5.png"  width="420"></img></a>
<br>

Vamos a scrapear el sitio de Latam para averiguar datos de vuelos en funcion el origen y destino, fecha y cabina. La información que esperamos obtener de cada vuelo es:
- Precio(s) disponibles
- Horas de salida, de llegada y duración
- Información de las escalas

¡Empecemos!

# Clase 10
Utilicemos lo aprendido hasta ahora para lograr el objetivo propuesto

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.latamairlines.com/es/es/ofertas-vuelos?origin=MAD&outbound=2023-04-21T12%3A00%3A00.000Z&destination=BOG&inbound=null&adt=1&chd=0&inf=0&trip=OW&cabin=Economy&redemption=false&sort=RECOMMENDED_A'
r = requests.get(url)

In [3]:
r.status_code

200

In [4]:
s = BeautifulSoup(r.text, 'lxml')
#print(s.prettify())

Vemos que la respuesta de la página no contiene la información que buscamos, ya que la misma aparece recién después de ejecutar el código JavaSCript que está en la respuesta.

In [5]:
with open('latam.html', 'w') as f:
    f.write(r.text)
    f.close

## Selenium

Selenium es una herramienta que nos permitirá controlar un navegador y podremos utilizar las funcionalidades del motor de JavaScript para cargar el contenido que no viene en el HTML de la página. Para esto necesitamos el módulo `webdriver`.

In [6]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

Paso 1: instanciar un **driver** del navegador

In [7]:
options = webdriver.ChromeOptions()
options.add_argument("--incognito")
driver = webdriver.Chrome(ChromeDriverManager().install(), options = options)

C:\Users\andre\AppData\Local\Temp\ipykernel_15904\698620076.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options = options)


Paso 2: hacer que el navegador cargue la página web.

In [8]:
driver.get(url)

Paso 3: extraer la información de la página

In [29]:
def obtener_precios(vuelo):
    """Funcion que retorna una lista de diccionarios con las distintas tarifas"""
    #Abrir Apartado    
    #vuelo.find_element(by =By.XPATH, value ='.//div[@role="button"]').click()
    
    tarifas = vuelo.find_elements(by =By.XPATH, value ='.//ol[@class="sc-bCMXmc KIHXT"]/li/*')

    precios = []
    for tarifa in tarifas:
        nombre = tarifa.find_element(by =By.XPATH, value ='.//span[@class="sc-buGlAa goFssn"]/..').text.replace('\n',' ')
        moneda = tarifa.find_elements(by =By.XPATH, value ='.//div[@class="sc-gCUMDz dqhYlE"]/span/*')[1].text
        valor = tarifa.find_elements(by =By.XPATH, value ='.//div[@class="sc-gCUMDz dqhYlE"]/span/*')[0].text
        
        dict_tarifa = {nombre:{'moneda':moneda, 'valor':valor}}
        precios.append(dict_tarifa)

        #print(dict_tarifa) 
    
    #vuelo.find_element(by =By.XPATH, value ='//button[@class="MuiButtonBase-root MuiButton-root MuiButton-text sc-kAzzGY jHprQs MuiButton-disableElevation"]').click()
    
    return precios

In [10]:
def obtener_tiempos(vuelo):
   """Función que a partir de un vuelo devuelve un diccionario con los tiempos
      de cada viaje por escala
   """
   tiempo_salida = vuelo.find_elements(by=By.XPATH, value='.//div[@class = "sc-ckYZGd cjlufk flight-information"]/span[@class = "sc-dXLFzO kxrBKe"]')[0].text
   tiempo_llegada = vuelo.find_elements(by=By.XPATH, value='.//div[@class = "sc-ckYZGd cjlufk flight-information"]/span[@class = "sc-dXLFzO kxrBKe"]')[1].text 
   duracion = vuelo.find_element(by=By.XPATH, value='.//div[@class = "sc-ckYZGd cjlufk flight-duration"]/span[@class = "sc-dAOnuy XwJVd"]').text 

   info_tiempos = {'t_salida': tiempo_salida, 't_llegada': tiempo_llegada, 'duracion': duracion}
    
   return info_tiempos

In [11]:
def obtener_datos_escalas(vuelo):
    """Funcion que retorna una lista de diccionarios con la informacion de las escalas de cada vuelo"""
    
    segmentos = vuelo.find_elements(by=By.XPATH, value = '//div[@class = "MuiDialogContent-root sc-bZQynM hnBdls"]//section[@class = "sc-hqGPoI daKcNu"]')
    escalas = vuelo.find_elements(by=By.XPATH, value = '//div[@class = "MuiDialogContent-root sc-bZQynM hnBdls"]//section[@class = "sc-hqGPoI dxlHTG"]')
    
    info_escalas = []
    for i,segmento in enumerate(segmentos):
        #Info Vuelo
        origen = segmento.find_elements(by=By.XPATH, value='.//div[@class = "iataCode"]/span')[0].text
        
        salida_origen = segmento.find_elements(by=By.XPATH, value='.//div[@class = "iataCode"]/span')[1].text
        
        destino = segmento.find_elements(by=By.XPATH, value='.//div[@class = "iataCode"]/span')[2].text
        
        llegada_destino = segmento.find_elements(by=By.XPATH, value='.//div[@class = "iataCode"]/span')[3].text
        
        duracion_vuelo_destino = segmento.find_elements(by=By.XPATH, value='.//span[@class="time"]')[1].text
        
        #Info Avion
        numero_vuelo = segmento.find_elements(by=By.XPATH, value='.//div[@class="sc-exdmVY hjDxso"]/div')[0].text
        modelo_avion_vuelo = segmento.find_elements(by=By.XPATH, value='.//div[@class="sc-exdmVY hjDxso"]/span')[0].text    
        
        #Info Escalas
        escala = "Si"
        if i == len(segmentos)-1:
            escala = "No"
            duracion_escala_destino = None
        else:
            duracion_escala_destino = escalas[i].find_elements(by=By.XPATH, value='.//span[@class="time"]')[0].text
            
        dict_vuelo = {f'vuelo_{i}':{'origen':origen,'hora_salida':salida_origen,'destino':destino,'hora_llegada':llegada_destino,'duracion_vuelo':duracion_vuelo_destino,
                                    'numero_vuelo':numero_vuelo,'modelo':modelo_avion_vuelo,'escala': escala,'duracion_escala':duracion_escala_destino}}
        
        #print(dict_vuelo)
        info_escalas.append(dict_vuelo)
        
    #Cerrar Apartado
    #vuelo.find_element(by =By.XPATH, value ='//button[@class = "MuiButtonBase-root MuiIconButton-root sc-ifAKCX kdVmSh Dialog__CloseButton Dialog__CloseButton--titled"]').click()    
    
    return info_escalas

In [31]:
def obtener_info(driver):
    
    vuelos = driver.find_elements(by=By.XPATH,value='//li[@class="sc-bvTASY cfqKKq"]')
    print(f'Se encontraron {len(vuelos)} vuelos.')
    print('Iniciando Scraping...')
    
    info = []
    for i,vuelo in enumerate(vuelos):
        #obtenemos los tiempos generados en cada vuelo
        tiempos = obtener_tiempos(vuelo)
        
        #Abrir Apartado    
        vuelo.find_element(by =By.XPATH, value ='.//div[@role="button"]').click()
        time.sleep(2)
        #obtenemos la informacion de los precios
        precios = obtener_precios(vuelo)    
        #Cerrar Apartado
        vuelo.find_element(by =By.XPATH, value ='//button[@class="MuiButtonBase-root MuiButton-root MuiButton-text sc-kAzzGY jHprQs MuiButton-disableElevation"]').click()
        time.sleep(2)
        
        #Abrir Apartado
        vuelo.find_element(by =By.XPATH, value =f'.//div[@class = "sc-dOkuiw flITYG"]/a').click()
        time.sleep(4)
        #obtenemos la infirnmacion de las escalas
        escalas = obtener_datos_escalas(vuelo)
        #Cerrar Apartado
        vuelo.find_element(by =By.XPATH, value ='//button[@class = "MuiButtonBase-root MuiIconButton-root sc-ifAKCX kdVmSh Dialog__CloseButton Dialog__CloseButton--titled"]').click()
        time.sleep(2)
        
        info.append({'precios':precios,'tiempo':tiempos, 'escalas':escalas})
        print(i)
    return info

In [ ]:
vuelo.find_element(by =By.XPATH, value =f'.//div[@class = "sc-dOkuiw flITYG"]/a').click()

For the moment we should accept cookies before running the whole script

In [32]:
a = obtener_info(driver)

Se encontraron 7 vuelos.
Iniciando Scraping...
0
1
2
3
4
5
6


In [33]:
a[3]

{'precios': [{'plus': {'moneda': 'EUR', 'valor': '424,12'}},
  {'plus Premium Business': {'moneda': 'EUR', 'valor': '1468,12'}},
  {'top Premium Business': {'moneda': 'EUR', 'valor': '3381,12'}}],
 'tiempo': {'t_salida': '12:05',
  't_llegada': '20:45',
  'duracion': '15 h 40 min'},
 'escalas': [{'vuelo_0': {'origen': 'MAD',
    'hora_salida': '12:05',
    'destino': 'UIO',
    'hora_llegada': '16:15',
    'duracion_vuelo': '11 h 10 min',
    'numero_vuelo': 'LA5425',
    'modelo': 'Airbus A350-900',
    'escala': 'Si',
    'duracion_escala': '2 h 49 min'}},
  {'vuelo_1': {'origen': 'UIO',
    'hora_salida': '19:04',
    'destino': 'BOG',
    'hora_llegada': '20:45',
    'duracion_vuelo': '1 h 41 min',
    'numero_vuelo': 'LA1442*',
    'modelo': 'Airbus A319',
    'escala': 'No',
    'duracion_escala': None}}]}

Paso 4: cerrar el navegador

In [34]:
driver.close()